LightGBM is a really convenient to use, fast to train and usually accurate implementation of boosted trees. Here I use optuna for hyperparameter search using Bayesian optimization methods, with 5-fold cross validation, to gain a fairly accurate model. Also I leverage some seemingly minor but very useful built in features of the LightGBM library to handle categorical variables.

In [1]:
##Import the required packages. These include pandas, numpy,scikit-learn and optuna

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import KFold
import optuna
import optuna.integration.lightgbm as lgb


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-mar-2021/train.csv
/kaggle/input/tabular-playground-series-mar-2021/test.csv


Read the train and test csv's into variables and list out the column names

In [2]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')

train.columns.to_list()

['id',
 'cat0',
 'cat1',
 'cat2',
 'cat3',
 'cat4',
 'cat5',
 'cat6',
 'cat7',
 'cat8',
 'cat9',
 'cat10',
 'cat11',
 'cat12',
 'cat13',
 'cat14',
 'cat15',
 'cat16',
 'cat17',
 'cat18',
 'cont0',
 'cont1',
 'cont2',
 'cont3',
 'cont4',
 'cont5',
 'cont6',
 'cont7',
 'cont8',
 'cont9',
 'cont10',
 'target']

Convert the categorical data into the category type such that lightgbm can handle the categorical variables. Unless you leverage learned embeddings for categorical variables, this fares better that one hot encoding or label encoding

In [3]:
conts = ['cont0','cont1','cont2','cont3','cont4','cont5','cont6','cont7','cont8','cont9','cont10']
cats = ['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9','cat10','cat11','cat12','cat13','cat14','cat15','cat16','cat17','cat18','target']
for c in train.columns:
    col_type = train[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        train[c] = train[c].astype('category')

for c in test.columns:
    col_type = test[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        test[c] = test[c].astype('category')

Specify dependent and independent variables and create a lgb Dataset object

In [4]:
X= train[['cont0','cont1','cont2','cont3','cont4','cont5','cont6','cont7','cont8','cont9','cont10','cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9','cat10','cat11','cat12','cat13','cat14','cat15','cat16','cat17','cat18']]
Y = train[['target']]

Use the optuna lightGBM integration to do hyperparamater optimization with 5 fold cross validation. Make sure to pass in the argument 'auto' for categorical_feature for automated feature engineering for categorical input features.

In [5]:
from sklearn.model_selection import StratifiedKFold
kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

Notice how I've specified auc as the metric.

In [6]:
dtrain = lgb.Dataset(X,Y,categorical_feature = 'auto')

params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
}

tuner = lgb.LightGBMTunerCV(
    params, dtrain, verbose_eval=100, early_stopping_rounds=1000000, folds=kfolds
)

tuner.run()

print("Best score:", tuner.best_score)
best_params = tuner.best_params
print("Best params:", best_params)
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))

[I 2021-03-01 20:41:52,415] A new study created in memory with name: no-name-92d6c35c-c991-489f-a2bd-e3394118cf66
feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]

[100]	cv_agg's auc: 0.892008 + 0.00103649
[200]	cv_agg's auc: 0.893159 + 0.00104752
[300]	cv_agg's auc: 0.893393 + 0.000974532
[400]	cv_agg's auc: 0.893529 + 0.000856763
[500]	cv_agg's auc: 0.89352 + 0.000842855
[600]	cv_agg's auc: 0.893431 + 0.000780106
[700]	cv_agg's auc: 0.89331 + 0.000789088
[800]	cv_agg's auc: 0.893118 + 0.000825148
[900]	cv_agg's auc: 0.892949 + 0.000867973
[1000]	cv_agg's auc: 0.892746 + 0.000884992


feature_fraction, val_score: 0.893550:  14%|#4        | 1/7 [01:58<11:48, 118.14s/it]

[100]	cv_agg's auc: 0.891533 + 0.000866496
[200]	cv_agg's auc: 0.892522 + 0.000915965
[300]	cv_agg's auc: 0.892623 + 0.000856064
[400]	cv_agg's auc: 0.892501 + 0.000911613
[500]	cv_agg's auc: 0.89234 + 0.000904109
[600]	cv_agg's auc: 0.892118 + 0.000948539
[700]	cv_agg's auc: 0.891841 + 0.000917707
[800]	cv_agg's auc: 0.891497 + 0.000939116
[900]	cv_agg's auc: 0.891255 + 0.000944222
[1000]	cv_agg's auc: 0.890921 + 0.000923366


feature_fraction, val_score: 0.893550:  29%|##8       | 2/7 [03:39<09:00, 108.02s/it]

...

[100]	cv_agg's auc: 0.894314 + 0.0010437
[200]	cv_agg's auc: 0.894985 + 0.00109764
[300]	cv_agg's auc: 0.894605 + 0.00107565
[400]	cv_agg's auc: 0.89389 + 0.00101422
[500]	cv_agg's auc: 0.893298 + 0.000959342
[600]	cv_agg's auc: 0.892588 + 0.000979304
[700]	cv_agg's auc: 0.891875 + 0.000978021
[800]	cv_agg's auc: 0.891241 + 0.000992385
[900]	cv_agg's auc: 0.890663 + 0.000988129
[1000]	cv_agg's auc: 0.890131 + 0.00101357


min_data_in_leaf, val_score: 0.895099:  80%|########  | 4/5 [12:29<03:07, 187.85s/it]

[100]	cv_agg's auc: 0.894331 + 0.00102328
[200]	cv_agg's auc: 0.894905 + 0.00103034
[300]	cv_agg's auc: 0.894535 + 0.000876282
[400]	cv_agg's auc: 0.893955 + 0.000865514
[500]	cv_agg's auc: 0.893233 + 0.000879131
[600]	cv_agg's auc: 0.892649 + 0.000844389
[700]	cv_agg's auc: 0.89197 + 0.000854087
[800]	cv_agg's auc: 0.891377 + 0.000819855
[900]	cv_agg's auc: 0.890865 + 0.000813573
[1000]	cv_agg's auc: 0.890334 + 0.000848413


min_data_in_leaf, val_score: 0.895099: 100%|##########| 5/5 [15:34<00:00, 186.98s/it]

Best score: 0.8950986360077963
Best params: {'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 7.959176411127531, 'lambda_l2': 5.381687699546818e-05, 'num_leaves': 135, 'feature_fraction': 0.4, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20}
  Params: 
    objective: binary
    metric: auc
    verbosity: -1
    boosting_type: gbdt
    feature_pre_filter: False
    lambda_l1: 7.959176411127531
    lambda_l2: 5.381687699546818e-05
    num_leaves: 135
    feature_fraction: 0.4
    bagging_fraction: 1.0
    bagging_freq: 0
    min_child_samples: 20


Inspect the best score for AUC value

In [7]:
tuner.best_score

0.8950986360077963

Assign the best params to a variable

In [8]:
params = tuner.best_params

Use these parameters to train a LightGBM model on the entire training dataset 


In [9]:
import lightgbm as lgb
id_test = test.id.to_list()
model = lgb.train(params, dtrain, num_boost_round=1000)

Predict on the test set and save file. Make sure you set index=False 

In [10]:
X_test = test[['cont0','cont1','cont2','cont3','cont4','cont5','cont6','cont7','cont8','cont9','cont10','cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9','cat10','cat11','cat12','cat13','cat14','cat15','cat16','cat17','cat18']]
preds = model.predict(X_test)
resultf = pd.DataFrame()
resultf['id'] = id_test
resultf['target'] = preds
resultf.to_csv('submission.csv',index=False)